## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Khatanga,71.9667,102.5000,-27.13,79,41,8.63,RU,scattered clouds
1,1,Mataura,-46.1927,168.8643,47.43,94,100,1.50,NZ,overcast clouds
2,2,Georgetown,5.4112,100.3354,84.00,83,20,2.30,MY,few clouds
3,3,Grand Gaube,-20.0064,57.6608,79.00,77,0,1.99,MU,clear sky
4,4,Hobart,-42.8794,147.3294,69.80,83,90,10.36,AU,light rain


In [35]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
selected_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
selected_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,Georgetown,5.4112,100.3354,84.00,83,20,2.30,MY,few clouds
3,3,Grand Gaube,-20.0064,57.6608,79.00,77,0,1.99,MU,clear sky
9,9,Kruisfontein,-34.0033,24.7314,78.01,62,0,7.00,ZA,clear sky
11,11,Diamantino,-14.4086,-56.4461,86.97,61,77,5.97,BR,broken clouds
16,16,Vaini,-21.2000,-175.2000,78.80,94,75,1.14,TO,broken clouds
18,18,Karratha,-20.7377,116.8463,81.18,78,17,8.68,AU,few clouds
19,19,Mount Isa,-20.7333,139.5000,84.20,61,40,10.36,AU,scattered clouds
21,21,Tautira,-17.7333,-149.1500,82.00,65,13,1.99,PF,few clouds
22,22,Rikitea,-23.1203,-134.9692,78.13,71,40,12.08,PF,scattered clouds
23,23,Puerto Ayora,-0.7393,-90.3518,84.20,65,75,19.57,EC,broken clouds


In [37]:
# 4a. Determine if there are any empty rows.
selected_cities_df.count()

City_ID                210
City                   210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Country                206
Current Description    210
dtype: int64

In [38]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=selected_cities_df.dropna()
preferred_cities_df.count()

City_ID                206
City                   206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Country                206
Current Description    206
dtype: int64

In [39]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Georgetown,MY,84.00,few clouds,5.4112,100.3354,
3,Grand Gaube,MU,79.00,clear sky,-20.0064,57.6608,
9,Kruisfontein,ZA,78.01,clear sky,-34.0033,24.7314,
11,Diamantino,BR,86.97,broken clouds,-14.4086,-56.4461,
16,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,
18,Karratha,AU,81.18,few clouds,-20.7377,116.8463,
19,Mount Isa,AU,84.20,scattered clouds,-20.7333,139.5000,
21,Tautira,PF,82.00,few clouds,-17.7333,-149.1500,
22,Rikitea,PF,78.13,scattered clouds,-23.1203,-134.9692,
23,Puerto Ayora,EC,84.20,broken clouds,-0.7393,-90.3518,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]   
   
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()


City                   206
Country                206
Max Temp               206
Current Description    206
Lat                    206
Lng                    206
Hotel Name             206
dtype: int64

In [47]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))